In [1]:
from ultralytics import YOLO

In [2]:
from ultralytics.yolo.v8.detect.predict import DetectionPredictor

In [3]:
import cv2

In [4]:
# Load the YOLO model
model = YOLO()


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.Conv                  [128

In [8]:
# Training parameters
INIT_LR = 1e-4
EPOCHS = 10
BS = 16
patience=50
imgsz=224

In [11]:
# Train the model
model.train(data ="data.yaml", epochs=EPOCHS, batch=BS)

Ultralytics YOLOv8.0.20  Python-3.10.0 torch-2.3.1+cpu CPU
yolo\engine\trainer: task=detect, mode=train, model=None, data=data.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=None, workspace=4, nms=False, lr0=0.01, lrf=0.01, momentum=0.93

In [13]:
#prediction 
!!yolo predict model="C:\Users\sreet\OneDrive\Documents\Hope AI\16.Deep Learning\moon-crater-detection.v10i.yolov8\runs\detect\train9\weights\best.pt"

["WARNING  'source' is missing. Using default 'source=https://ultralytics.com/images/bus.jpg'.",
 "2024-07-21 15:40:31.703996: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found",
 '2024-07-21 15:40:31.704033: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.',
 'Ultralytics YOLOv8.0.20  Python-3.10.0 torch-2.3.1+cpu CPU',
 'YOLOv8n summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs',
 'Found https://ultralytics.com/images/bus.jpg locally at bus.jpg',
 'image 1/1 C:\\Users\\sreet\\OneDrive\\Documents\\Hope AI\\16.Deep Learning\\moon-crater-detection.v10i.yolov8\\bus.jpg: 640x480 197.1ms',
 'Speed: 0.0ms pre-process, 197.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)']

In [14]:
import cv2
import random
from ultralytics import YOLO

# Load the YOLO model
model = YOLO(r'C:\Users\sreet\OneDrive\Documents\Hope AI\16.Deep Learning\moon-crater-detection.v10i.yolov8\runs\detect\train9\weights\best.pt')

# Define a list of colors (one for each class)
colors = {}

# Open the webcam
cap = cv2.VideoCapture(0)

# Create a named window and set it to a smaller size
cv2.namedWindow("Webcam", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("Webcam", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_NORMAL)

# Resize the window to 75% of the screen size
screen_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) * 0.75)
screen_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) * 0.75)
cv2.resizeWindow("Webcam", screen_width, screen_height)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform prediction
    results = model(frame)

    # Process the results
    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0]
            cls = int(box.cls[0])

            # Assign a color to the class if not already assigned
            if cls not in colors:
                colors[cls] = [random.randint(0, 255) for _ in range(3)]

            # Get the color for the current class
            color = colors[cls]

            # Draw rectangle with a thicker border
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)

            # Put label with a larger font size and thicker text
            label = f'{model.names[cls]} {conf:.2f}'
            font_scale = 0.8
            font_thickness = 2

            # Determine the text size to create a background rectangle
            (text_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)
            cv2.rectangle(frame, (x1, y1 - text_height - 10), (x1 + text_width, y1), color, -1)
            
            # Put the text on the background rectangle
            cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness)

    # Display the frame
    cv2.imshow("Webcam", frame)

    # Print out predictions for debugging
    print(results)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close the windows
cap.release()
cv2.destroyAllWindows()

Ultralytics YOLOv8.0.20  Python-3.10.0 torch-2.3.1+cpu CPU
YOLOv8n summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
type: <class 'torch.Tensor'>
shape: torch.Size([0, 6])
dtype: torch.float32
 + tensor([], size=(0, 6))]
[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
type: <class 'torch.Tensor'>
shape: torch.Size([0, 6])
dtype: torch.float32
 + tensor([], size=(0, 6))]
[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
type: <class 'torch.Tensor'>
shape: torch.Size([0, 6])
dtype: torch.float32
 + tensor([], size=(0, 6))]
[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
type: <class 'torch.Tensor'>
shape: torch.Size([0, 6])
dtype: torch.float32
 + tensor([], size=(0, 6))]
[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
type: <class 'torch.Tensor'>
shape: torch.Size([0, 6])
dtype: torch.float32
 + tensor([], size=(0, 6))]
[Ultralytics YOLO <class 'ultralytics.yolo.engine.results.Boxes'> masks
type: <class 'torch.Tensor'>
shape: torch.Size([